In [1]:
import arcgis
import getpass
import pandas as pd
from arcgis.gis import GIS

In [2]:
password = getpass.getpass("Enter your password: ")

Enter your password: ········


In [3]:
gis = GIS("https://www.arcgis.com", "jjguo1993", password)

## Import Data From Annual_Parking_Study_Data.csv

In [ ]:
APSD = pd.read_csv("../Datasets/Annual_Parking_Study_Data.csv")

In [ ]:
APSD.info()

In [ ]:
APSD.head()

##### Select the 'Elmnkey' Column

In [ ]:
Elmntkey = APSD[['Elmntkey']].drop_duplicates()

In [ ]:
Elmntkey.head()

In [ ]:
Elmntkey.columns = ['ELMNTKEY']

In [ ]:
Elmntkey.info()

## Import Data From Blockface.csv

In [ ]:
Blockface = pd.read_csv("../Datasets/Blockface.csv")

In [ ]:
Blockface.head()

In [ ]:
Blockface.info()

In [ ]:
Blockface_slice = Blockface[['ELMNTKEY','SEGKEY']]

In [ ]:
Blockface_slice.head()

In [ ]:
Blockface.info()

In [ ]:
Blockface_slice['ELMNTKEY'] = Blockface_slice['ELMNTKEY'].astype('float64')

In [ ]:
Blockface_slice.dtypes

### Select the rows that the 'ELMNTKEY' value is in Elmnkey

In [ ]:
# ba_inner = pd.merge(Elmntkey, Blockface_slice, how='inner', on=['ELMNTKEY'])
ba_inner = Blockface_slice.loc[Blockface_slice['ELMNTKEY'].isin(Elmntkey['ELMNTKEY'])]

In [ ]:
ba_inner.head()

In [ ]:
ba_inner.dropna(inplace=True)

In [ ]:
ba_inner.info()

In [ ]:
SEGKEY = ba_inner[['SEGKEY']]
SEGKEY.head()

In [ ]:
SEGKEY.columns = ['COMPKEY']

## Compare the csv file and the data from shapefile

In [ ]:
Seattle_Streets = pd.read_csv("../Datasets/Seattle_Streets.csv")

In [ ]:
Seattle_Streets.head()

## Search the uploaded content and import the spatial dataframe

In [ ]:
streets = gis.content.search(query="title: Seattle Streets AND owner:Transportation_SeattleCityGIS",
                             item_type="Feature Layer", outside_org=True)

In [ ]:
streets

In [ ]:
streets=streets[0]

In [4]:
a = gis.map("Seattle", 10)

In [5]:
a

MapView(layout=Layout(height='400px', width='100%'), zoom=10.0)

In [ ]:
sdf = pd.DataFrame.spatial.from_layer(streets)

In [ ]:
sdf.info()

In [ ]:
sdf.dtypes

In [ ]:
sdf.spatial.validate()

In [ ]:
sdf.spatial.geometry_type

In [ ]:
sdf.head().spatial.geometry_type

## Select the rows again

In [ ]:
# street_block_merge = pd.merge(sdf,SEGKEY, how='inner', on=['COMPKEY'])
SEGKEY.head()
street_block_merge = sdf.loc[sdf['COMPKEY'].isin(SEGKEY['COMPKEY'])]

In [ ]:
street_block_merge.dropna(inplace=True)
# street_block_merge.spatial.validate()

In [ ]:
street_block_merge.spatial.geometry_type

In [ ]:
street_block_merge.info()

## Draw on the map

In [ ]:
Seattle = gis.map("Seattle", 10)
Seattle

In [ ]:
street_block_merge.spatial.plot(map_widget = Seattle, symbol_type='simple', colors='autumn')